In [ ]:
def ExonChaining(G, n):
    # G is a list of tuples, each tuple contains three elements: left endpoint, right endpoint, and weight
    s = [0] * (2 * n + 1)

    endpoints = sorted(set([v for interval in G for v in interval[:2]]))  # get all endpoints and remove duplicates
    endpoint_index = {v: i+1 for i, v in enumerate(endpoints)}  
    
    intervals = [(endpoint_index[l], endpoint_index[r], w) for l, r, w in G]
    for i in range(1, len(endpoints) + 1):
        right_intervals = [interval for interval in intervals if interval[1] == i]        
        if right_intervals:
            for l, r, w in right_intervals:
                s[i] = max(s[i-1], s[l] + w)

        else:
            s[i] = s[i - 1]
    return s

G = [
    (1, 5, 5),   
    (2, 3, 3),   
    (4, 8, 6),   
    (6, 12, 10),  
    (7, 17, 12),   
    (9, 10, 1),  
    (11, 15, 7),  
    (13, 14, 0),  
    (16, 18, 4)  
]

G2=[
    (1, 5, 4),   
    (2, 3, 5),   
    (4, 8, 11),   
    (6, 12, 8),  
    (7, 17, 9),   
    (9, 10, 0),  
    (11, 15, 6),  
    (13, 14, 3),  
    (16, 18, 2)  
]

result = ExonChaining(G2, len(G2))
print(result)

[0, 0, 0, 5, 5, 5, 5, 5, 16, 16, 16, 16, 16, 16, 19, 22, 22, 22, 24]


In [47]:
def ExonChaining(G, n):
    # G 是一个包含区间元组的列表，每个元组为 (左端点, 右端点, 权重)
    # n 是区间数量
    # 初始化动态规划数组和选择的区间记录
    s = [0] * (2 * n + 1)
    selected_intervals = [[] for _ in range(2 * n + 1)]  # 用于记录每个端点的选择区间
    
    # 创建所有端点的列表，并为其进行排序
    endpoints = sorted(set([v for interval in G for v in interval[:2]]))  # 获取去重后的所有端点
    endpoint_index = {v: i+1 for i, v in enumerate(endpoints)}  # 为每个端点分配索引值
    
    # 初始化区间表，记录每个区间的左、右端点索引和权重
    intervals = [(endpoint_index[l], endpoint_index[r], w) for l, r, w in G]
    
    # 遍历所有端点
    for i in range(1, len(endpoints) + 1):
        # 查找当前端点是否是某个区间的右端点
        right_intervals = [interval for interval in intervals if interval[1] == i]
        
        if right_intervals:
            # 遍历所有以该端点为右端点的区间
            for l, r, w in right_intervals:
                # 比较是否选择当前区间，取最大值
                s[i] = max(s[i-1], s[l] + w)
                # 如果选择了当前区间，更新选择的区间记录
                if s[i] == s[l] + w:
                    selected_intervals[i] = selected_intervals[l] + [(endpoints[l-1], endpoints[r-1], w)]
        else:
            # 如果没有选择任何区间，则继承前一个端点的选择和权重
            s[i] = s[i - 1]
            selected_intervals[i] = selected_intervals[i - 1]
    
    # 返回最优解的最大权重和选择的区间
    return s, selected_intervals[len(endpoints)]


# 定义区间 (左端点, 右端点, 权重)
G = [
    (1, 5, 5),   
    (2, 3, 3),   
    (4, 8, 6),   
    (6, 12, 10),  
    (7, 17, 12),   
    (9, 10, 1),  
    (11, 15, 7),  
    (13, 14, 0),  
    (16, 18, 4)  
]
G2=[
    (1, 5, 4),   
    (2, 3, 5),   
    (4, 8, 11),   
    (6, 12, 8),  
    (7, 17, 9),   
    (9, 10, 0),  
    (11, 15, 6),  
    (13, 14, 3),  
    (16, 18, 2)  
]
s, selected = ExonChaining(G2, len(G2))
print("最大不重叠区间的总权重为:", s)
print("选择的区间为:", selected)

最大不重叠区间的总权重为: [0, 0, 0, 5, 5, 5, 5, 5, 16, 16, 16, 16, 16, 16, 19, 22, 22, 22, 24]
选择的区间为: [(2, 3, 5), (4, 8, 11), (9, 10, 0), (11, 15, 6), (16, 18, 2)]


In [3]:
from Bio import SeqIO

def read_fasta(file):
    sequences = []
    for record in SeqIO.parse(file, "fasta"):
        sequences.append(str(record.seq))
    return sequences

sequences = read_fasta("APOBEC3A.fasta")


def load_blosum62(file):
    blosum62 = {}
    with open(file) as f:
        lines = f.readlines()
        amino_acids = lines[0].split() 
        for line in lines[1:]:
            parts = line.split()
            row_acid = parts[0]  
            scores = list(map(int, parts[1:]))  
            blosum62[row_acid] = dict(zip(amino_acids, scores))
    return blosum62
def needleman_wunsch(seq1, seq2, gap_penalty, blosum62):
    n = len(seq1)
    m = len(seq2)
    score_matrix = [[0] * (m + 1) for _ in range(n + 1)]

    for i in range(1, n + 1):
        score_matrix[i][0] = gap_penalty * i
    for j in range(1, m + 1):
        score_matrix[0][j] = gap_penalty * j

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match = score_matrix[i-1][j-1] + blosum62[seq1[i-1]][seq2[j-1]]
            delete = score_matrix[i-1][j] + gap_penalty
            insert = score_matrix[i][j-1] + gap_penalty
            score_matrix[i][j] = max(match, delete, insert)

    return score_matrix[n][m]

In [5]:
blosum62 = load_blosum62("BLOSUM62.txt")
gap_penalty = -5
sequences = read_fasta("APOBEC3A.fasta")

ans=(0,1)
res=-1
for i in range(len(sequences)):
    for j in range(i + 1, len(sequences)):
        score = needleman_wunsch(sequences[i], sequences[j], gap_penalty, blosum62)
        if score>res:
            res=score
            ans=(i,j)
        # print(f"Similarity between sequence {i+1} and {j+1}: {score}")

print(f"The most similar sequences are {ans[0]+1} and {ans[1]+1} with a similarity score of {res}")


The most similar sequences are 8 and 10 with a similarity score of 1103
